In [1]:
import numpy as np
import pandas as pd
import subprocess
import re

In [2]:
task = "gemma"
mem = "64"
cpus = "1"
inpath = "../calls/"
covar = "gemma_covar.cov"
maf = "0.05"
prefix = "final_gwas_dataset"
subprocess.run(["sbatch --mem=" + mem + "g --ntasks=" + 
     cpus + " ../" + task + ".sh " + 
     inpath + " " + covar + " " + maf + " " + prefix ], shell=True)

Submitted batch job 9028995


CompletedProcess(args=['sbatch --mem=64g --ntasks=1 ../gemma.sh ../calls/ gemma_covar.cov 0.05 final_gwas_dataset'], returncode=0)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [10]:
df = pd.read_csv("../calls/output/final_gwas_dataset.assoc.txt", sep = '\t')
df['minuslog10pvalue'] = -np.log10(df["p_lrt"])
df = df[~df["p_lrt"].isna()].reset_index(drop=True)

In [5]:
df["pval_cor"] = scipy.stats.false_discovery_control(df["p_lrt"].to_numpy())

In [6]:
df = df[["ps", "minuslog10pvalue", "chr"]]

In [ ]:
df.chr = df.chr.astype('category')
#df.chr = df.chr.cat.set_categories(['ch-%i' % i for i in range(12)], ordered=True)
df = df.sort_values('chr')

# How to plot gene vs. -log10(pvalue) and colour it by chromosome?
df['ind'] = range(len(df))
df_grouped = df.groupby(('chr'))

# manhattan plot
fig = plt.figure(figsize=(14, 8)) # Set the figure size
ax = fig.add_subplot(111)
colors = ['darkred','darkgreen','darkblue', 'gold']
x_labels = []
x_labels_pos = []
for num, (name, group) in enumerate(df_grouped):
    group.plot(kind='scatter', x='ind', y='minuslog10pvalue',color=colors[num % len(colors)], ax=ax)
    x_labels.append(name)
    x_labels_pos.append((group['ind'].iloc[-1] - (group['ind'].iloc[-1] - group['ind'].iloc[0])/2))
ax.set_xticks(x_labels_pos)
ax.set_xticklabels(x_labels)

# set axis limits
ax.set_xlim([0, len(df)])
ax.set_ylim([0, 9])

# x axis label
ax.set_xlabel('Chromosome')
plt.xticks(rotation=90)
# show the graph
plt.show()